In [36]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_three.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Use the following context to answer the questions. If you don't know the answer, don't say you don't know and don't wonder.:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(f"질문: {question}")
    print(f"답변: {result.content}")

questions = [
    "Is Aaronson guilty?",
    "What message did he write in the table?",
    "Who is Julia?",
    "내가 2번째로 질문한 내용이 뭐지?"
]

for question in questions:
    invoke_chain(question)

질문: Is Aaronson guilty?
답변: Yes, according to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.
질문: What message did he write in the table?
답변: He wrote "2+2=5" in the dust on the table.
질문: Who is Julia?
답변: Julia is a character in the text who is significant to the protagonist. She is someone Winston loves and cares for deeply.
질문: 내가 2번째로 질문한 내용이 뭐지?
답변: 두 번째 질문은 "테이블에 어떤 메시지를 썼습니까?" 였습니다.
